<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/hybridhestonhullwhiteprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
import QuantLib as ql
import unittest
import math

# Helper to mimic TopLevelFixture
class QuantLibTestCase(unittest.TestCase):
    def setUp(self):
        self.saved_settings = ql.SavedSettings()
        # Set a default evaluation date if tests rely on it
        self.evaluation_date = ql.Date(15, ql.May, 2020) # Example fixed date
        ql.Settings.instance().evaluationDate = self.evaluation_date


    def tearDown(self):
        self.saved_settings = None # Restores settings

def flatRate(evaluationDate, rate, dayCounter):
    return ql.YieldTermStructureHandle(ql.FlatForward(evaluationDate, rate, dayCounter))

def flatVol(evaluationDate, vol, dayCounter):
    return ql.BlackVolTermStructureHandle(
        ql.BlackConstantVol(evaluationDate, ql.NullCalendar(), vol, dayCounter)
    )

class HybridHestonHullWhiteProcessTests(QuantLibTestCase):

    def testBsmHullWhiteEngine(self):
        self.subTestName = ("Testing European option pricing for a BSM process "
                            "with one-factor Hull-White model...")
        # print(self.subTestName)

        dc = ql.Actual365Fixed()
        today = ql.Settings.instance().evaluationDate
        maturity = today + ql.Period(20, ql.Years) # Long maturity

        spot_q = ql.RelinkableQuoteHandle(ql.SimpleQuote(100.0))
        q_rate_q = ql.SimpleQuote(0.04)
        qTS = flatRate(today, q_rate_q.value(), dc)
        r_rate_q = ql.SimpleQuote(0.0525)
        rTS = flatRate(today, r_rate_q.value(), dc)
        vol_q = ql.SimpleQuote(0.25)
        volTS = flatVol(today, vol_q.value(), dc)

        hullWhiteModel = ql.HullWhite(rTS, 0.00883, 0.00526) # a, sigma
        bsmProcess = ql.BlackScholesMertonProcess(spot_q, qTS, rTS, volTS)
        exercise = ql.EuropeanExercise(maturity)

        # ATM Forward Payoff
        fwd = spot_q.value() * qTS.discount(maturity) / rTS.discount(maturity)
        payoff = ql.PlainVanillaPayoff(ql.Option.Call, fwd)
        option = ql.EuropeanOption(payoff, exercise)

        tol = 1e-8
        correlations = [-0.75, -0.25, 0.0, 0.25, 0.75]
        expected_vols = [0.217064577, 0.243995801, 0.256402830, 0.268236596, 0.290461343]

        for i, corr in enumerate(correlations):
            bsm_hw_engine = ql.AnalyticBSMHullWhiteEngine(corr, bsmProcess, hullWhiteModel)
            option.setPricingEngine(bsm_hw_engine)
            npv_hybrid = option.NPV()
            delta_hybrid = option.delta()
            gamma_hybrid = option.gamma()
            theta_hybrid = option.theta() # Per day
            vega_hybrid = option.vega()

            # Comparison with a BS model with the expected equivalent vol
            comp_vol_ts = flatVol(today, expected_vols[i], dc)
            bs_process_comp = ql.BlackScholesMertonProcess(spot_q, qTS, rTS, comp_vol_ts)
            bs_engine_comp = ql.AnalyticEuropeanEngine(bs_process_comp)

            comp_option = ql.EuropeanOption(payoff, exercise)
            comp_option.setPricingEngine(bs_engine_comp)
            npv_comp = comp_option.NPV()
            delta_comp = comp_option.delta()
            gamma_comp = comp_option.gamma()
            theta_comp = comp_option.theta() # Per day
            vega_comp = comp_option.vega()

            # Check implied vol from hybrid NPV matches expected vol
            implied_vol = ql.Settings.instance().includeReferenceDateEvents = True
            try:
                 implied_vol = comp_option.impliedVolatility(npv_hybrid, bs_process_comp, 1e-10, 100, 1e-12, 5.0)
            finally:
                 ql.Settings.instance().includeReferenceDateEvents = False


            self.assertAlmostEqual(implied_vol, expected_vols[i], delta=tol,
                                   msg=f"Implied vol mismatch for corr={corr}. Calc: {implied_vol}, Exp: {expected_vols[i]}")

            self.assertAlmostEqual(npv_comp / npv_hybrid if abs(npv_hybrid)>1e-9 else 1.0, 1.0, delta=tol,
                                   msg=f"NPV mismatch for corr={corr}. Hybrid: {npv_hybrid}, CompBS: {npv_comp}")
            self.assertAlmostEqual(delta_comp, delta_hybrid, delta=tol,
                                   msg=f"Delta mismatch for corr={corr}. Hybrid: {delta_hybrid}, CompBS: {delta_comp}")

            # Relative check for gamma, theta, vega if NPV is not too small
            if abs(npv_hybrid) > 1e-4 :
                self.assertAlmostEqual(gamma_comp / npv_hybrid, gamma_hybrid / npv_hybrid, delta=tol,
                                    msg=f"Gamma mismatch for corr={corr}.")
                self.assertAlmostEqual(theta_comp / npv_hybrid, theta_hybrid / npv_hybrid, delta=tol,
                                    msg=f"Theta mismatch for corr={corr}.")
                self.assertAlmostEqual(vega_comp / npv_hybrid, vega_hybrid / npv_hybrid, delta=tol,
                                    msg=f"Vega mismatch for corr={corr}.")
            else: # Absolute check for small NPVs
                self.assertAlmostEqual(gamma_comp, gamma_hybrid, delta=tol*10, msg=f"Gamma mismatch (abs) for corr={corr}.")
                self.assertAlmostEqual(theta_comp, theta_hybrid, delta=tol*10, msg=f"Theta mismatch (abs) for corr={corr}.")
                self.assertAlmostEqual(vega_comp, vega_hybrid, delta=tol*10, msg=f"Vega mismatch (abs) for corr={corr}.")


    def testCompareBsmHWandHestonHW(self):
        self.subTestName = ("Comparing European option pricing for a BSM process "
                            "with one-factor Hull-White model...")
        # print(self.subTestName)
        dc = ql.Actual365Fixed()
        today = ql.Settings.instance().evaluationDate

        dates_yc = []
        rates_yc, div_rates_yc = [], []
        for i in range(41): # 0 to 40 years
            dates_yc.append(today + ql.Period(i, ql.Years))
            rates_yc.append(0.01 + 0.0002 * math.exp(math.sin(i / 4.0)))
            div_rates_yc.append(0.02 + 0.0001 * math.exp(math.sin(i / 5.0)))

        s0_q = ql.RelinkableQuoteHandle(ql.SimpleQuote(100.0))
        rTS = ql.YieldTermStructureHandle(ql.ZeroCurve(dates_yc, rates_yc, dc))
        qTS = ql.YieldTermStructureHandle(ql.ZeroCurve(dates_yc, div_rates_yc, dc))

        vol_init_val = 0.25
        vol_q = ql.SimpleQuote(vol_init_val)
        volTS = flatVol(today, vol_q.value(), dc)

        bsmProcess = ql.BlackScholesMertonProcess(s0_q, qTS, rTS, volTS)
        # Heston params to mimic BS: v0=theta=vol^2, sigma_v very small
        hestonProcess = ql.HestonProcess(
            rTS, qTS, s0_q,
            vol_q.value()**2, 1.0, vol_q.value()**2, 1e-4, 0.0
        )
        hestonModel = ql.HestonModel(hestonProcess)
        hullWhiteModel = ql.HullWhite(rTS, 0.01, 0.01) # a, sigma

        bsm_hw_engine = ql.AnalyticBSMHullWhiteEngine(0.0, bsmProcess, hullWhiteModel) # rho_sr = 0
        # N integration points for Heston-HW
        heston_hw_engine = ql.AnalyticHestonHullWhiteEngine(hestonModel, hullWhiteModel, 128)

        tol = 1e-5
        strikes_mult = [0.25, 0.5, 0.75, 0.8, 0.9, 1.0, 1.1, 1.2, 1.5, 2.0, 4.0]
        maturities_years = [1, 2, 3, 5, 10, 15, 20, 25, 30]
        option_types = [ql.Option.Put, ql.Option.Call]

        for opt_type in option_types:
            for strike_mult in strikes_mult:
                for mat_yr in maturities_years:
                    maturityDate = today + ql.Period(mat_yr, ql.Years)
                    exercise = ql.EuropeanExercise(maturityDate)

                    # Strike is relative to forward in C++ test. Let's assume S0*strike_mult.
                    # C++: Real fwd = j * spot->value() * qTS->discount(maturityDate) / rTS->discount(maturityDate);
                    # This means j is interpreted as K/Fwd_adj where Fwd_adj uses S0 as base.
                    # For simplicity here, let's use strike_mult as a direct multiplier to S0,
                    # or interpret j as moneyness K/S0 if that was the intent.
                    # The C++ uses `j * spot->value()` scaled by discount factors, which effectively sets
                    # the strike to be `j` times the forward value based on `spot->value()`.
                    # If `j=1.0`, it's ATM-Forward.

                    # Calculate forward price for this maturity
                    fwd_price = s0_q.value() * qTS.discount(maturityDate) / rTS.discount(maturityDate)
                    strike_val = strike_mult * fwd_price # K = moneyness_factor * Fwd

                    payoff = ql.PlainVanillaPayoff(opt_type, strike_val)
                    option = ql.EuropeanOption(payoff, exercise)

                    option.setPricingEngine(bsm_hw_engine)
                    calculated_bsmhw = option.NPV()

                    option.setPricingEngine(heston_hw_engine)
                    expected_hestonhw = option.NPV()

                    # C++ test: std::fabs(calculated-expected) > calculated*tol && std::fabs(calculated-expected) > tol
                    # This means if calculated is small, use absolute tol, otherwise relative.
                    diff_val = abs(calculated_bsmhw - expected_hestonhw)
                    if diff_val > tol and diff_val > abs(calculated_bsmhw) * tol :
                        self.fail(
                            f"NPV mismatch: BSM-HW vs Heston-HW. Type={opt_type}, K/F={strike_mult}, Mat={mat_yr}yr. "
                            f"BSM-HW: {calculated_bsmhw:.6f}, Heston-HW: {expected_hestonhw:.6f}, Diff: {diff_val:.2e}"
                        )


    def testZeroBondPricing(self):
        self.subTestName = "Testing Monte-Carlo zero bond pricing..."
        # print(self.subTestName)
        dc = ql.Actual360()
        today = ql.Settings.instance().evaluationDate

        dates_yc, times_yc, rates_yc = [today], [0.0], [0.02]
        for i in range(120, 240): # 10 to 20 years in months
            dt_obj = today + ql.Period(i, ql.Months)
            dates_yc.append(dt_obj)
            rates_yc.append(0.02 + 0.0002 * math.exp(math.sin(i / 8.0)))
            times_yc.append(dc.yearFraction(today, dt_obj))

        maturity_final_zb = dates_yc[-1] + ql.Period(10, ql.Years)
        dates_yc.append(maturity_final_zb)
        rates_yc.append(0.04)
        times_yc.append(dc.yearFraction(today, maturity_final_zb))

        s0_q = ql.RelinkableQuoteHandle(ql.SimpleQuote(100.0))
        ts = ql.YieldTermStructureHandle(ql.ZeroCurve(dates_yc, rates_yc, dc))
        ds_heston = flatRate(today, 0.0, dc) # Dividend yield for Heston part

        hestonProcess = ql.HestonProcess(ts, ds_heston, s0_q, 0.02, 1.0, 0.2, 0.5, -0.8)
        # HullWhiteForwardProcess: a, sigma. Forward measure time needs to be set.
        hwFwdProcess = ql.HullWhiteForwardProcess(ts, 0.05, 0.05)
        hwFwdProcess.setForwardMeasureTime(dc.yearFraction(today, maturity_final_zb))

        hwModel = ql.HullWhite(ts, 0.05, 0.05) # For analytic bond option

        # HybridHestonHullWhiteProcess(hestonProc, hwProc, equityShortRateCorr, disc=Euler)
        jointProcess = ql.HybridHestonHullWhiteProcess(hestonProcess, hwFwdProcess, -0.4)

        # Use times_yc up to the point before the final maturity_final_zb for the grid
        # C++: TimeGrid grid(times.begin(), times.end()-1); implies excluding the last point.
        time_grid_values = times_yc[:-1]
        grid = ql.TimeGrid(time_grid_values) # Pass list of times

        # MC Simulation
        # C++: SobolBrownianBridgeRsg
        # Python: ql.SobolBrownianGenerator with bridge=True? Or SobolBrownianBridgeRsg directly if wrapped.
        # Let's use SobolBrownianGenerator which is common. Bridge might be default or specific.
        # Number of factors = jointProcess.factors()
        # Number of steps = grid.size() - 1

        # Factors: Heston (S,v) -> 2, HW (r) -> 1. Total = 3.
        # Corr matrix for Brownian motions W_s, W_v, W_r.
        # jointProcess.correlation() should give the correlation structure if needed.
        # Let's assume jointProcess.factors() gives the correct number for path generator.

        factors = jointProcess.factors()
        steps = grid.size() -1 # Number of time steps in the grid

        # SobolBrownianGenerator(factors, steps, SobolBrownianGenerator.Diagonal, seed, ordering, seed_skip)
        # C++ SobolBrownianBridgeRsg(factors, steps)
        # For MultiPathGenerator, a BrownianGeneratorFactory might be easier.
        # Let's use a simple pseudo-random generator for now if Sobol setup is complex.
        # Or try Sobol directly if its constructor matches.
        # QL Python's MultiPathGenerator requires a Process and a TimeGrid.
        # And a generator (e.g., from a factory).

        # For SobolBrownianBridgeRsg, it implies a specific sequence generator.
        # Let's try with a standard Sobol generator factory.
        # The C++ uses rsg_type directly. Python may need factory.
        generator_factory = ql.SobolBrownianGeneratorFactory(
             ql.SobolBrownianGenerator.Diagonal, # ordering
             1234, # seed, not used by Sobol typically
             ql.SobolRsg.JoeKuoD7 # directionIntegers
        )
        # For Sobol, typically pass number of paths for generator.
        # MultiPathGenerator(process, timeGrid, generatorFactory, antithetic)
        # Or MultiPathGenerator(process, timeGrid, Rsg Traits based generator, antithetic)
        # The C++ test uses the Rsg directly.
        # Let's create the specific RSG type as in C++.
        # pyql.SobolBrownianBridgeRsg is not directly wrapped.
        # We'll use the factory route for `MultiPathGenerator`.

        # Simpler path: Use PseudoRandom for MultiPathGenerator in Python for now
        # If specific SobolBridgeRsg is needed, that's a deeper binding check.
        generator = ql.MultiPathGenerator(jointProcess, grid, generator_factory, False)


        num_paths = 8191
        num_maturities_to_check = 90 # 'm' in C++
        option_tenor_steps = 24 # Time steps for option maturity

        # Initialize stats collectors (Python list of GeneralStatistics)
        zero_stats = [ql.GeneralStatistics() for _ in range(num_maturities_to_check)]
        option_stats = [ql.GeneralStatistics() for _ in range(num_maturities_to_check)]

        for i in range(num_paths):
            sample_path = generator.next()
            path_values = sample_path.value # path_values[component_idx][time_idx]

            for j in range(1, num_maturities_to_check): # Start from 1 as in C++
                t_idx = j # Index in grid/path
                t_time = grid[t_idx]

                # Ensure t_idx + option_tenor_steps is within grid bounds
                if t_idx + option_tenor_steps >= grid.size():
                    continue

                T_time = grid[t_idx + option_tenor_steps]

                # States at time t_time (index t_idx in path)
                # Path components: 0=Stock (logS), 1=Variance (v), 2=Rate (r)
                # jointProcess state order: S, v, r
                # path.value from MultiPathGenerator typically gives:
                # path.value[0] -> Heston S component (often logS)
                # path.value[1] -> Heston v component
                # path.value[2] -> HW r component

                # The process->numeraire expects array [S, v, r]
                # Path from HybridHestonHullWhiteProcess:
                # [0] is log(S_t)
                # [1] is v_t
                # [2] is r_t (short rate from HW)

                states_at_t = ql.Array(3)
                states_at_t[0] = math.exp(path_values[0][t_idx]) # S_t = exp(logS_t)
                states_at_t[1] = path_values[1][t_idx]          # v_t
                states_at_t[2] = path_values[2][t_idx]          # r_t

                # Zero bond price P(t,T_final_numeraire) = 1 / numeraire(t, states_at_t)
                # The numeraire for HybridHestonHullWhiteProcess is B(t) = exp(integral_0^t r_s ds)
                # So, 1.0 / jointProcess.numeraire(t_time, states_at_t) is P(0,t) in domestic currency.
                # This seems to be the discount factor from t to today (0).
                # The C++ code seems to interpret 1/numeraire as P(t, T_num_final_date) if numeraire is defined as B(t)/B(T_num_final_date).
                # Let's assume numeraire(t, states) gives the value of the money market account at time t, B(t).
                # Then 1.0 / numeraire would be P(0,t).
                # However, the C++ context of jointProcess->numeraire is often related to forward measure pricing.
                # Given hwFwdProcess->setForwardMeasureTime(maturity_final_zb), the numeraire is likely P(t, maturity_final_zb).
                # So, 1.0/jointProcess->numeraire() would be 1/P(t, maturity_final_zb), not a ZCB P(0,t).

                # Let's use the HullWhite model to get P(t, T_option_maturity) using r_t from path.
                # P(t,T_option_maturity) where r(t) = states_at_t[2]
                # But the states array is [S,v,r]. jointProcess->numeraire should use these.
                # The path gives r_t. Discount factor P(t, T_final) should be what numeraire represents.
                # C++ `1.0/jointProcess->numeraire(t, states)` is the ZCB maturing at forward measure time,
                # priced at time `t` given `states`.
                # The ZB in `zeroStat` is P(t, T_numeraire_final_date).
                # The ZB in `ts->discount(t)` is P(0, t). These are different.
                # The test probably checks consistency of MC numeraire with analytic numeraire for that HW process.

                # This test is subtle. The numeraire in HybridHestonHullWhiteProcess is typically the
                # money market account B(t) or a ZCB P(t, T_fixed_num_maturity).
                # If hwFwdProcess.setForwardMeasureTime(maturity_final_zb) sets the numeraire to P(t, maturity_final_zb),
                # then 1.0/numeraire gives 1/P(t, maturity_final_zb), which is not P(0,t).
                # The comparison `expected = ts->discount(t)` means P(0,t).
                # This implies the MC simulation should be for P(0,t) somehow.

                # Let's assume `1.0/jointProcess->numeraire(t, states)` is indeed P(0,t) under simulation measure.
                # This happens if the process is simulated under risk-neutral measure and numeraire is B(t).

                # If numeraire is B(t):
                discount_to_t = 1.0 / jointProcess.numeraire(t_time, states_at_t)
                zero_stats[j].add(discount_to_t)

                # Zero bond option: Option on P(T, T_option_maturity) struck at `strike_zb_opt`.
                # Value at t: P(0,t) * E_t[ max(0, P(T, T_option_maturity) - K_zb) / P(0,T) ]
                # Or, if pricing in T-forward measure: P(0,T) * E_t^T[ max(0, P(T, T_option_maturity) - K_zb) ]
                # The C++ formula `zeroBond * std::max(0.0, hwModel->discountBond(t, T, states[2])-strike)`
                # suggests `zeroBond` is P(0,t), and `hwModel->discountBond(t,T,r_t)` is P(t,T) given r_t.
                # This is a standard ZB option pricing formula under spot measure.
                strike_zb_opt = 0.5
                # hwModel->discountBond(startTime, endTime, rateAtStartTime)
                # Here, startTime=t_time, endTime=T_time, rateAtStartTime=states_at_t[2] (r_t)
                price_P_t_T = hwModel.discountBond(t_time, T_time, states_at_t[2])

                option_payoff_at_t = max(0.0, price_P_t_T - strike_zb_opt)
                option_value_at_0 = discount_to_t * option_payoff_at_t
                option_stats[j].add(option_value_at_0)


        for j in range(1, num_maturities_to_check):
            t_time = grid[j]

            calculated_zb = zero_stats[j].mean()
            expected_zb = ts.discount(t_time) # P(0,t) from input curve
            self.assertAlmostEqual(calculated_zb, expected_zb, delta=0.03,
                                   msg=f"Zero bond price mismatch at t={t_time:.2f}. Calc: {calculated_zb:.4f}, Exp: {expected_zb:.4f}")

            if j + option_tenor_steps >= grid.size(): continue
            T_time = grid[j + option_tenor_steps]

            calculated_opt = option_stats[j].mean()
            # hwModel.discountBondOption(type, strike, optionExpiry, bondMaturity, rateAtOptionExpiry=Null)
            # Here, rateAtOptionExpiry is not given, so it uses current yield curve for forward rate.
            # The MC uses the simulated r_t.
            # To match, the analytic formula for ZB option also needs to be forward-starting from t=0.
            # P(0,t) * E_t[ max(P(t,T)-K,0) ].
            # The hwModel.discountBondOption prices as of today (t=0).
            # Option expiry is t_time, bond maturity is T_time.
            expected_opt = hwModel.discountBondOption(ql.Option.Call, strike_zb_opt, t_time, T_time)

            self.assertAlmostEqual(calculated_opt, expected_opt, delta=0.0035,
                                   msg=(f"Zero bond option price mismatch for option on P({t_time:.2f},{T_time:.2f}). "
                                        f"Calc: {calculated_opt:.4f}, Exp: {expected_opt:.4f}"))


    def testMcVanillaPricing(self):
        self.subTestName = "Testing Monte-Carlo vanilla option pricing..."
        # print(self.subTestName)
        dc = ql.Actual360()
        today = ql.Settings.instance().evaluationDate

        dates_yc, rates_yc, div_rates_yc = [], [], []
        for i in range(41):
            dates_yc.append(today + ql.Period(i, ql.Years))
            rates_yc.append(0.03 + 0.0003 * math.exp(math.sin(i / 4.0)))
            div_rates_yc.append(0.02 + 0.0001 * math.exp(math.sin(i / 5.0)))

        maturity_date = today + ql.Period(20, ql.Years) # Very long maturity

        s0_q = ql.RelinkableQuoteHandle(ql.SimpleQuote(100.0))
        rTS = ql.YieldTermStructureHandle(ql.ZeroCurve(dates_yc, rates_yc, dc))
        qTS = ql.YieldTermStructureHandle(ql.ZeroCurve(dates_yc, div_rates_yc, dc))

        vol_init_val = 0.25
        vol_q = ql.SimpleQuote(vol_init_val)
        volTS_bsm = flatVol(today, vol_q.value(), dc)

        bsmProcess = ql.BlackScholesMertonProcess(s0_q, qTS, rTS, volTS_bsm)
        # Heston params for near-BS behavior: sigma_v very small.
        hestonProcess = ql.HestonProcess(
            rTS, qTS, s0_q, 0.0625, 0.5, 0.0625, 1e-5, 0.3 # v0, kappa, theta, sigma_v, rho_sv
        )
        hwFwdProcess = ql.HullWhiteForwardProcess(rTS, 0.01, 0.01) # a, sigma_r
        hwFwdProcess.setForwardMeasureTime(dc.yearFraction(today, maturity_date))

        tol_mc = 0.05 # Absolute tolerance for MC engine
        correlations_sr = [-0.9, -0.5, 0.0, 0.5, 0.9] # Equity-Rate correlation
        strikes_test = [100.0]

        for corr_sr in correlations_sr:
            for strike_val in strikes_test:
                # Hybrid process with equity-rate correlation `corr_sr`
                # rho_sv from hestonProcess (0.3) is Heston's S-v correlation.
                jointProcess = ql.HybridHestonHullWhiteProcess(hestonProcess, hwFwdProcess, corr_sr)

                payoff = ql.PlainVanillaPayoff(ql.Option.Put, strike_val)
                exercise = ql.EuropeanExercise(maturity_date)

                optionHestonHW = ql.EuropeanOption(payoff, exercise)
                # MCHestonHullWhiteEngine uses traits for generator
                # C++: MakeMCHestonHullWhiteEngine<PseudoRandom>(...)
                # Python: MCHestonHullWhiteEngine(process, antithetic, control, requiredTolerance, seed, Sobol=False)
                engine_mc = ql.MCHestonHullWhiteEngine(
                    jointProcess,
                    timeSteps=1, # Single step for Euler over full period (as in C++)
                    antitheticVariate=True,
                    controlVariate=True,
                    requiredTolerance=tol_mc,
                    seed=42,
                    Sobol=False # PseudoRandom
                )
                optionHestonHW.setPricingEngine(engine_mc)

                # Reference: AnalyticBSMHullWhiteEngine
                hwModel_ref = ql.HullWhite(rTS, hwFwdProcess.a(), hwFwdProcess.sigma())
                optionBsmHW_ref = ql.EuropeanOption(payoff, exercise)
                engine_bsmhw_ref = ql.AnalyticBSMHullWhiteEngine(corr_sr, bsmProcess, hwModel_ref)
                optionBsmHW_ref.setPricingEngine(engine_bsmhw_ref)

                calculated_mc = optionHestonHW.NPV()
                error_mc = optionHestonHW.errorEstimate()
                expected_bsmhw = optionBsmHW_ref.NPV()

                # C++ conditions:
                # (i != 0.0 && std::fabs(calculated - expected) > 3 * error) ||
                # (i == 0.0 && std::fabs(calculated - expected) > 1e-4)
                # i is corr_sr here.

                if corr_sr != 0.0:
                    self.assertLessEqual(abs(calculated_mc - expected_bsmhw), 3 * error_mc,
                                         msg=(f"MC HestonHW vs BSMHW (corr != 0) failed. Corr={corr_sr}, K={strike_val}. "
                                              f"MC: {calculated_mc:.4f} +/- {error_mc:.4f}, BSMHW: {expected_bsmhw:.4f}"))
                else: # corr_sr == 0.0, stricter absolute check
                    self.assertAlmostEqual(calculated_mc, expected_bsmhw, delta=1e-4,
                                           msg=(f"MC HestonHW vs BSMHW (corr == 0) failed. Corr={corr_sr}, K={strike_val}. "
                                                f"MC: {calculated_mc:.4f}, BSMHW: {expected_bsmhw:.4f}"))


    def testMcPureHestonPricing(self):
        self.subTestName = "Testing Monte-Carlo Heston option pricing..."
        # print(self.subTestName)
        dc = ql.Actual360()
        today = ql.Settings.instance().evaluationDate

        dates_yc, rates_yc, div_rates_yc = [], [], []
        for i in range(101): # 0 to 100 months
            dates_yc.append(today + ql.Period(i, ql.Months))
            rates_yc.append(0.02 + 0.0002 * math.exp(math.sin(i / 10.0)))
            div_rates_yc.append(0.02 + 0.0001 * math.exp(math.sin(i / 20.0)))

        maturity_date = today + ql.Period(2, ql.Years)

        s0_q = ql.RelinkableQuoteHandle(ql.SimpleQuote(100.0))
        rTS = ql.YieldTermStructureHandle(ql.ZeroCurve(dates_yc, rates_yc, dc))
        qTS = ql.YieldTermStructureHandle(ql.ZeroCurve(dates_yc, div_rates_yc, dc))

        # Heston process for pure Heston pricing
        hestonProcess_pure = ql.HestonProcess(
            rTS, qTS, s0_q, 0.08, 1.5, 0.0625, 0.5, -0.8 # v0,k,th,sig,rho
        )
        # HW process with tiny sigma_r to make it deterministic-like for interest rates
        # This is to test if MCHestonHullWhiteEngine can recover pure Heston prices
        # when HW part is negligible and rho_sr = 0.
        hwFwdProcess_negligible = ql.HullWhiteForwardProcess(rTS, 0.1, 1e-8) # a, sigma_r
        # Set forward measure time far out, matters less if sigma_r is tiny
        hwFwdProcess_negligible.setForwardMeasureTime(dc.yearFraction(today, maturity_date + ql.Period(1, ql.Years)))

        tol_mc_abs = 0.001 # Absolute tolerance for MC engine
        # Correlations for equity-rate (rho_sr). Heston's rho_sv is -0.8.
        correlations_sr = [-0.45, 0.45, 0.25]
        strikes_test = [100.0, 75.0, 50.0, 150.0]

        for corr_sr in correlations_sr:
            for strike_val in strikes_test:
                # Hybrid process: hestonProcess_pure, hwFwdProcess_negligible, corr_sr
                # C++: HybridHestonHullWhiteProcess::Euler discretization
                jointProcess_hybrid = ql.HybridHestonHullWhiteProcess(
                    hestonProcess_pure, hwFwdProcess_negligible, corr_sr,
                    ql.HybridHestonHullWhiteProcess.Euler # Discretization scheme
                )

                payoff = ql.PlainVanillaPayoff(ql.Option.Put, strike_val)
                exercise = ql.EuropeanExercise(maturity_date)

                optionHestonHW_mc = ql.EuropeanOption(payoff, exercise)
                engine_mc_hybrid = ql.MCHestonHullWhiteEngine(
                    jointProcess_hybrid,
                    timeSteps=2, # C++ test uses 2 steps
                    antitheticVariate=True,
                    controlVariate=True,
                    requiredTolerance=tol_mc_abs,
                    seed=42,
                    Sobol=False # PseudoRandom
                )
                optionHestonHW_mc.setPricingEngine(engine_mc_hybrid)

                # Reference: Pure Analytic Heston
                optionPureHeston_ref = ql.EuropeanOption(payoff, exercise)
                engine_heston_analytic = ql.AnalyticHestonEngine(ql.HestonModel(hestonProcess_pure))
                optionPureHeston_ref.setPricingEngine(engine_heston_analytic)

                expected_pure_heston = optionPureHeston_ref.NPV()
                calculated_mc_hybrid = optionHestonHW_mc.NPV()
                error_mc_hybrid = optionHestonHW_mc.errorEstimate()

                # C++ condition: std::fabs(calculated - expected) > 3*error && std::fabs(calculated - expected) > tol
                # This means fail if |diff| > 3*err AND |diff| > abs_tol_mc
                # If MC can't reach abs_tol_mc within 3*error_estimate, it's a failure.
                diff_val = abs(calculated_mc_hybrid - expected_pure_heston)
                if diff_val > 3 * error_mc_hybrid and diff_val > tol_mc_abs:
                     self.fail(
                        f"MC Hybrid (near pure Heston) vs Analytic Heston failed. Corr_sr={corr_sr}, K={strike_val}. "
                        f"MC: {calculated_mc_hybrid:.4f} +/- {error_mc_hybrid:.4f}, AnalyticHeston: {expected_pure_heston:.4f}, Diff: {diff_val:.2e}"
                    )

    # ... testAnalyticHestonHullWhitePricing, testCallableEquityPricing, etc.
    # These would follow a similar translation pattern.
    # testCallableEquityPricing is interesting due to manual path generation and payoff logic.

    @unittest.skip("Extensive manual path generation and payoff logic.")
    def testCallableEquityPricing(self):
        self.subTestName = "Testing the pricing of a callable equity product..."
        # This test involves manual Monte Carlo simulation logic:
        # - Setting up a Schedule and TimeGrid.
        # - Iterating paths from MultiPathGenerator.
        # - Applying a custom callable payoff structure within the loop.
        # - Calculating numeraire adjustments.
        # This requires careful replication of the C++ loop and financial logic.
        pass

    def testFdmHestonHullWhiteEngine(self):
        self.subTestName = "Testing the FDM Heston Hull-White engine..."
        # print(self.subTestName)

        today = ql.Date(28, ql.March, 2004)
        ql.Settings.instance().evaluationDate = today
        exerciseDate = ql.Date(28, ql.March, 2012) # 8 years
        dc = ql.Actual365Fixed()

        s0_q = ql.RelinkableQuoteHandle(ql.SimpleQuote(100.0))
        rTS = flatRate(today, 0.05, dc)
        qTS = flatRate(today, 0.02, dc)

        vol_init = 0.30
        volTS_bsm = flatVol(today, vol_init, dc)
        v0_heston = vol_init**2

        # Heston process for near-BS behavior (sigma_v very small)
        hestonProcess_fdm = ql.HestonProcess(
            rTS, qTS, s0_q, v0_heston, 1.0, v0_heston, 0.000001, 0.0
        )
        hestonModel_fdm = ql.HestonModel(hestonProcess_fdm)

        # BSM process for reference AnalyticBSMHullWhiteEngine
        bsmProcess_ref = ql.BlackScholesMertonProcess(s0_q, qTS, rTS, volTS_bsm)

        # Hull-White process and model
        hwProcess_fdm = ql.HullWhiteProcess(rTS, 0.00883, 0.01) # a, sigma_r
        hwModel_ref = ql.HullWhite(rTS, hwProcess_fdm.a(), hwProcess_fdm.sigma())

        exercise = ql.EuropeanExercise(exerciseDate)
        correlations_sr = [-0.85, 0.5]
        strikes_test = [75.0, 120.0, 160.0]

        for corr_sr in correlations_sr:
            for strike_val in strikes_test:
                payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike_val)
                option = ql.EuropeanOption(payoff, exercise)

                # FDM Heston-HW Engine
                # FdHestonHullWhiteVanillaEngine(hestonModel, hwProcess, equityShortRateCorr,
                #                                tGrid, xGrid, vGrid, rGrid, ...)
                # C++: 50, 200, 10, 15 for t,x,v,r grids
                engine_fdm_hhw = ql.FdHestonHullWhiteVanillaEngine(
                    hestonModel_fdm, hwProcess_fdm, corr_sr,
                    50, 200, 10, 15 # t, x, v, r grids
                )
                option.setPricingEngine(engine_fdm_hhw)
                calculated_npv = option.NPV()
                calculated_delta = option.delta()
                calculated_gamma = option.gamma()

                # Analytic BSM-HW Engine for reference
                engine_analytic_bsmhw = ql.AnalyticBSMHullWhiteEngine(
                    corr_sr, bsmProcess_ref, hwModel_ref
                )
                option.setPricingEngine(engine_analytic_bsmhw)
                expected_npv = option.NPV()
                expected_delta = option.delta()
                expected_gamma = option.gamma()

                self.assertAlmostEqual(calculated_npv, expected_npv, delta=0.01,
                                       msg=f"FDM HHW NPV mismatch. Corr={corr_sr}, K={strike_val}. FDM: {calculated_npv:.4f}, Analytic: {expected_npv:.4f}")
                self.assertAlmostEqual(calculated_delta, expected_delta, delta=0.001,
                                       msg=f"FDM HHW Delta mismatch. Corr={corr_sr}, K={strike_val}. FDM: {calculated_delta:.4f}, Analytic: {expected_delta:.4f}")
                self.assertAlmostEqual(calculated_gamma, expected_gamma, delta=0.001,
                                       msg=f"FDM HHW Gamma mismatch. Corr={corr_sr}, K={strike_val}. FDM: {calculated_gamma:.4f}, Analytic: {expected_gamma:.4f}")


    # HestonModelData, HullWhiteModelData, SchemeData, VanillaOptionData are structs for test cases
    # These will be Python dictionaries or lists of dictionaries.

    # testBsmHullWhitePricing (convergence speed)
    # testSpatialDiscretizationError (spatial convergence)
    # These are more involved due to iterating over many model params and schemes.
    # The HestonHullWhiteCorrelationConstraint is a custom C++ class.
    # Its `test` method would need to be a Python callable if used in calibration.

    # testHestonHullWhiteCalibration is a very complex calibration test.
    # It involves:
    # 1. Setting up a target volatility surface (from a known HHW model).
    # 2. Pre-calibrating Heston params using AnalyticHestonEngine and HestonModelHelper.
    # 3. Then, calibrating the Heston part of HHW using FdHestonHullWhiteVanillaEngine.
    # 4. Using a custom HestonHullWhiteCorrelationConstraint.
    # This is a significant piece of work to translate accurately.

    def testH1HWPricingEngine(self):
        self.subTestName = "Testing the H1-HW approximation engine..."
        # print(self.subTestName)

        today = ql.Date(15, ql.July, 2012)
        ql.Settings.instance().evaluationDate = today
        exerciseDate = ql.Date(13, ql.July, 2022) # 10 years
        dc = ql.Actual365Fixed()
        exercise = ql.EuropeanExercise(exerciseDate)
        s0_q = ql.RelinkableQuoteHandle(ql.SimpleQuote(100.0))

        r_rate, q_rate = 0.02, 0.00
        v0_h, theta_h, kappa_v_h = 0.05, 0.05, 0.3
        # sigma_v_h varies
        rho_sv, rho_sr = -0.30, 0.6
        kappa_r_hw, sigma_r_hw = 0.01, 0.01

        rTS = flatRate(today, r_rate, dc)
        qTS = flatRate(today, q_rate, dc)

        # BS process for implied vol calculation
        flatVolTS_bs = flatVol(today, 0.20, dc) # Dummy vol for BS process
        bsProcess_for_iv = ql.GeneralizedBlackScholesProcess(s0_q, qTS, rTS, flatVolTS_bs)

        # Hull-White model for H1HW engine
        hwProcess_h1 = ql.HullWhiteProcess(rTS, kappa_r_hw, sigma_r_hw) # Not used directly by engine, but params are
        hullWhiteModel_h1 = ql.HullWhite(rTS, kappa_r_hw, sigma_r_hw)

        tol_iv = 0.0001
        sigmas_v_test = [0.3, 0.6]
        strikes_test = [40.0, 80.0, 100.0, 120.0, 180.0]
        # expected[sigma_v_idx][strike_idx]
        expected_ivs = [
            [0.267503, 0.235742, 0.228223, 0.223461, 0.217855],
            [0.263626, 0.211625, 0.199907, 0.193502, 0.190025]
        ]

        for j_sig, sigma_v_val in enumerate(sigmas_v_test):
            hestonProcess_h1 = ql.HestonProcess(
                rTS, qTS, s0_q, v0_h, kappa_v_h, theta_h, sigma_v_val, rho_sv
            )
            hestonModel_h1 = ql.HestonModel(hestonProcess_h1)

            for i_k, strike_val in enumerate(strikes_test):
                payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike_val)
                option = ql.EuropeanOption(payoff, exercise)

                # AnalyticH1HWEngine(hestonModel, hullWhiteModel, equityShortRateCorr, N_integration_points)
                engine_h1hw = ql.AnalyticH1HWEngine(
                    hestonModel_h1, hullWhiteModel_h1, rho_sr, 144
                )
                option.setPricingEngine(engine_h1hw)
                npv_h1hw = option.NPV()

                implied_vol_h1hw = ql.Settings.instance().includeReferenceDateEvents = True
                try:
                    implied_vol_h1hw = option.impliedVolatility(npv_h1hw, bsProcess_for_iv, 1e-8, 200, 1e-9, 4.0)
                finally:
                     ql.Settings.instance().includeReferenceDateEvents = False

                self.assertAlmostEqual(implied_vol_h1hw, expected_ivs[j_sig][i_k], delta=tol_iv,
                                       msg=(f"H1HW Implied Vol mismatch. sigma_v={sigma_v_val}, K={strike_val}. "
                                            f"Calc IV: {implied_vol_h1hw:.6f}, Exp IV: {expected_ivs[j_sig][i_k]:.6f}"))


if __name__ == '__main__':
    print("Running QuantLib-Python HybridHestonHullWhiteProcessTests...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)